### The goal of this scraper to scape the sales table table from carsalesdatabase and load tables to mysql workbench
 * base_url: http://carsalesbase.com/us-car-sales-data/

In [1]:
# Import dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import time
from selenium import webdriver

from selenium.webdriver.support.ui import Select
driver = webdriver.Chrome()
from selenium.webdriver.common.by import By
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
from mysql_conn import password
import datetime
import os

### If make and model is predefined, use csv

In [2]:
table_links = pd.read_csv('database/table_links.csv')

table_links.head()

,make,model,Link
0,Chevrolet,Colorado,http://carsalesbase.com/us-car-sales-data/chev...
1,Jeep,Wrangler,http://carsalesbase.com/us-car-sales-data/jeep...
2,Chevrolet,Silverado,http://carsalesbase.com/us-car-sales-data/chev...
3,Subaru,WRX,http://carsalesbase.com/us-car-sales-data/suba...
4,GMC,Canyon,http://carsalesbase.com/us-car-sales-data/gmc/...


### Loop thru tables in each Link and capture tables
* Need to store each table as browser loops thru
* Add code to export as csv

#### First test one car in list to confirm code is working as expected

In [3]:
test_table = table_links[0:1]
test_table

,make,model,Link
0,Chevrolet,Colorado,http://carsalesbase.com/us-car-sales-data/chev...


In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

for i in test_table['Link']:
    
    try:
        link = i
        print(f'Processing data for: {link}')
        browser.visit(link)
        time.sleep(5)
        html = browser.html
        #response = requests.get(url)
        soup = bs(html, 'lxml')
        tables = pd.read_html(html)
            

        
    
    except (KeyError):

        print("Data unavilable...skipping.")

Processing data for: http://carsalesbase.com/us-car-sales-data/chevrolet/chevrolet-colorado


In [7]:
# call table 0-4 to test
yearly_table = tables[0]
yearly_table

,0,1,2,3,4
0,NaN,Chevrolet Colorado 2015,Chevrolet Colorado 2016,Chevrolet Colorado 2017,Chevrolet Colorado 2018
1,January,5.942,5.508,6.413,8.011
2,February,6.563,7.394,7.519,8.050
3,March,6.621,9.718,8.426,12.798
4,April,7.010,10.362,9.221,NaN
5,May,8.881,9.196,9.091,NaN
6,June,6.558,9.049,9.631,NaN
7,July,7.209,9.195,11.206,NaN
8,August,7.114,9.242,10.256,NaN
9,September,7.334,10.383,11.271,NaN


## Test Clean 

In [8]:
# take 1 of 4 in list
yearly_table.columns = tables[0].iloc[0]

# fill null values
yearly_table_2 = yearly_table.fillna('empty')

# Create Month Column
yearly_table_2.rename(columns={ yearly_table_2.columns[0]: "Month"}, inplace=True)

# Capture make_name by stripping off year
make_name = yearly_table_2.columns[1][:-4].strip()

# Remove duplicate row
yearly_final = yearly_table_2.iloc[2:]

# Create 'Make Column'
yearly_final.insert(loc = 0, column = 'Make', value = make_name, allow_duplicates=False)

# Create empty header list varible
header = []

# Create new column headers using years (2015,2016,2017,etc.)
new_columns = yearly_final.columns[2:].values
for column in new_columns:
    header.append(column[-4:])
    
# Create new dataframe with year columns and values
yearly = yearly_final.rename(columns={ yearly_final.columns[2]: header[0],
                                      yearly_final.columns[3]: header[1],
                                      yearly_final.columns[4]:header[2],
                                      yearly_final.columns[5]:header[3]
                                      
                                  })
# make a copy dataframe copy
year_sale = yearly
# replace decimal point values with comma to denote thousands (U.S. units of cars sold)
year_sale['2015'] = [x.replace('.', ',') for x in year_sale['2015']]
year_sale['2016'] = [x.replace('.', ',') for x in year_sale['2016']]
year_sale['2017'] = [x.replace('.', ',') for x in year_sale['2017']]
year_sale['2018'] = [x.replace('.', ',') for x in year_sale['2018']]

year_sale

,Make,Month,2015,2016,2017,2018
2,Chevrolet Colorado,February,"6,563","7,394","7,519","8,050"
3,Chevrolet Colorado,March,"6,621","9,718","8,426","12,798"
4,Chevrolet Colorado,April,"7,010","10,362","9,221",null
5,Chevrolet Colorado,May,"8,881","9,196","9,091",null
6,Chevrolet Colorado,June,"6,558","9,049","9,631",null
7,Chevrolet Colorado,July,"7,209","9,195","11,206",null
8,Chevrolet Colorado,August,"7,114","9,242","10,256",null
9,Chevrolet Colorado,September,"7,334","10,383","11,271",null
10,Chevrolet Colorado,October,"7,059","10,578","9,990",null
11,Chevrolet Colorado,November,"6,230","8,669","10,346",null


In [25]:
file_name = make_name.strip()

file_name

'Chevrolet Colorado'

In [27]:
year_sale.to_csv(f'sales-csvs\{file_name}.csv',index=False,sep=',', na_rep='',encoding='utf-8')

## Call exported csvs to test

In [40]:
chevy_col = pd.read_csv('sales-csvs/Chevrolet Colorado.csv')
chevy_col = chevy_col.replace('empty', '')
chevy_col.head()

,Make,Month,2015,2016,2017,2018
0,Chevrolet Colorado,February,"6,563","7,394","7,519","8,050"
1,Chevrolet Colorado,March,"6,621","9,718","8,426","12,798"
2,Chevrolet Colorado,April,"7,010","10,362","9,221",
3,Chevrolet Colorado,May,"8,881","9,196","9,091",
4,Chevrolet Colorado,June,"6,558","9,049","9,631",


In [41]:
chevy_col.dtypes

Make     object
Month    object
2015     object
2016     object
2017     object
2018     object
dtype: object

## Single Link processing

In [8]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)



tables = []
link = 'http://carsalesbase.com/us-car-sales-data/jeep/jeep-wrangler'

print(f'Processing data for: {link}')
browser.visit(link)
time.sleep(5)
html = browser.html
#response = requests.get(url)
soup = bs(html, 'lxml')
tables = pd.read_html(html)
# take 1 of 4 in list
yearly_table = tables[0]
yearly_table.columns = tables[0].iloc[0]

# fill null values
yearly_table_2 = yearly_table.fillna('empty')

# Create Month Column
yearly_table_2.rename(columns={ yearly_table_2.columns[0]: "Month"}, inplace=True)

# Capture make_name by stripping off year
make_name = yearly_table_2.columns[1][:-4].strip()

# Remove duplicate row
yearly_final = yearly_table_2.iloc[2:]

# Create 'Make Column'
yearly_final.insert(loc = 0, column = 'Make', value = make_name, allow_duplicates=False)

# Create empty header list varible
header = []

# Create new column headers using years (2015,2016,2017,etc.)
new_columns = yearly_final.columns[2:].values
for column in new_columns:
    header.append(column[-4:])

# Create new dataframe with year columns and values
yearly = yearly_final.rename(columns={ yearly_final.columns[2]: header[0],
                                      yearly_final.columns[3]: header[1],
                                      yearly_final.columns[4]:header[2],
                                      yearly_final.columns[5]:header[3]

                                  })
# make a copy dataframe copy
year_sale = yearly
# replace decimal point values with comma to denote thousands (U.S. units of cars sold)
year_sale['2015'] = [x.replace('.', ',') for x in year_sale['2015']]
year_sale['2016'] = [x.replace('.', ',') for x in year_sale['2016']]
year_sale['2017'] = [x.replace('.', ',') for x in year_sale['2017']]
year_sale['2018'] = [x.replace('.', ',') for x in year_sale['2018']]

file_name = make_name.strip()
year_sale.to_csv(f'sales-csvs\{file_name}.csv',index=False,sep=',', na_rep='',encoding='utf-8')
tables = []



Processing data for: http://carsalesbase.com/us-car-sales-data/jeep/jeep-wrangler


In [9]:
tables[0]

,nan,Jeep Wrangler 2015,Jeep Wrangler 2016,Jeep Wrangler 2017,Jeep Wrangler 2018,Jeep Wrangler 2019
0,NaN,Jeep Wrangler 2015,Jeep Wrangler 2016,Jeep Wrangler 2017,Jeep Wrangler 2018,Jeep Wrangler 2019
1,January,11.683,10.797,11.334,11.739,13.024
2,February,12.911,13.234,13.641,15.936,15.001
3,March,17.524,17.710,16.336,27.829,NaN
4,April,18.849,19.003,18.841,29.776,NaN
5,May,22.324,19.551,19.931,25.102,NaN
6,June,19.159,20.060,18.839,23.110,NaN
7,July,19.320,18.741,18.698,21.308,NaN
8,August,18.160,15.290,16.808,20.168,NaN
9,September,17.583,14.255,15.714,15.983,NaN


## Test Load to mysql database

#### End one car test

### Looping through entire list and exporting tables as csvs

In [19]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
links = [
'http://carsalesbase.com/us-car-sales-data/chevrolet/chevrolet-colorado',
'http://carsalesbase.com/us-car-sales-data/jeep/jeep-wrangler',
'http://carsalesbase.com/us-car-sales-data/chevrolet/chevrolet-silverado',
'http://carsalesbase.com/us-car-sales-data/subaru/wrx-sti/',
'http://carsalesbase.com/us-car-sales-data/gmc/gmc-canyon',
'http://carsalesbase.com/us-car-sales-data/toyota/toyota-4runner',
'http://carsalesbase.com/us-car-sales-data/gmc/gmc-sierra',
'http://carsalesbase.com/us-car-sales-data/toyota/toyota-tacoma',
'http://carsalesbase.com/us-car-sales-data/honda/honda-ridgeline',
'http://carsalesbase.com/us-car-sales-data/toyota/toyota-tundra',
'http://carsalesbase.com/us-car-sales-data/bmw/bmw-7-series',
'http://carsalesbase.com/us-car-sales-data/fiat/fiat-500/',
'http://carsalesbase.com/us-car-sales-data/jaguar/jaguar-xj',
#'http://carsalesbase.com/us-car-sales-data/jaguar/jaguar-xk',
#'http://carsalesbase.com/us-car-sales-data/lincoln/lincoln-mks',
#'http://carsalesbase.com/us-car-sales-data/mercedes-benz/mercedes-benz-cl/',
'http://carsalesbase.com/us-car-sales-data/nissan/nissan-leaf',
'http://carsalesbase.com/us-car-sales-data/smart/smart-fortwo-electric-drive/'
]
tables = []

for i in links[5:]:
    
    try:
        tables = pd.DataFrame()
        link = i
        print(f'Processing data for: {link}')
        browser.visit(link)
        browser.visit(link)
        time.sleep(5)
        html = browser.html
        #response = requests.get(url)
        soup = bs(html, 'lxml')
        time.sleep(5)
        tables = pd.read_html(html)
        # take 1 of 4 in list
        yearly_table = tables[0]
        yearly_table.columns = tables[0].iloc[0]

        # fill null values
        yearly_table_2 = yearly_table.fillna('0')

        # Create Month Column
        yearly_table_2.rename(columns={ yearly_table_2.columns[0]: "Month"}, inplace=True)

        # Capture make_name by stripping off year
        make_name = yearly_table_2.columns[1][:-4].strip()

        # Remove duplicate row
        yearly_final = yearly_table_2.iloc[2:]

        # Create 'Make Column'
        yearly_final.insert(loc = 0, column = 'Make', value = make_name, allow_duplicates=False)

        # Create empty header list varible
        header = []

        # Create new column headers using years (2015,2016,2017,etc.)
        new_columns = yearly_final.columns[2:].values
        for column in new_columns:
            header.append(column[-4:])

        # Create new dataframe with year columns and values
        yearly = yearly_final.rename(columns={ yearly_final.columns[2]: header[0],
                                              yearly_final.columns[3]: header[1],
                                              yearly_final.columns[4]:header[2],
                                              yearly_final.columns[5]:header[3]

                                          })
        # make a copy dataframe copy
        year_sale = yearly
        # replace decimal point values with comma to denote thousands (U.S. units of cars sold)
        year_sale['2015'] = [x.replace('.', ',') for x in year_sale['2015']]
        year_sale['2016'] = [x.replace('.', ',') for x in year_sale['2016']]
        year_sale['2017'] = [x.replace('.', ',') for x in year_sale['2017']]
        year_sale['2018'] = [x.replace('.', ',') for x in year_sale['2018']]
        
        file_name = make_name.strip()
        year_sale.to_csv(f'sales-csvs/{file_name}.csv',index=False,sep=',', na_rep='',encoding='utf-8')
        print(f'file name: {file_name}')
        print(f'make name: {make_name}')
        file_name = ''
        make_name = ''
        tables = pd.DataFrame()
        yearly_table = pd.DataFrame()
        yearly_table_2 = pd.DataFrame()
        yearly = pd.DataFrame()
        yearly_final = ''
        year_sale = pd.DataFrame()


        
        
    
    except (KeyError):

        print("Data unavilable...skipping.")

Processing data for: http://carsalesbase.com/us-car-sales-data/toyota/toyota-4runner
file name: Toyota 4Runner
make name: Toyota 4Runner
Processing data for: http://carsalesbase.com/us-car-sales-data/gmc/gmc-sierra
file name: GMC Sierra
make name: GMC Sierra
Processing data for: http://carsalesbase.com/us-car-sales-data/toyota/toyota-tacoma
file name: Toyota Tacoma
make name: Toyota Tacoma
Processing data for: http://carsalesbase.com/us-car-sales-data/honda/honda-ridgeline
file name: Honda Ridgeline
make name: Honda Ridgeline
Processing data for: http://carsalesbase.com/us-car-sales-data/toyota/toyota-tundra
file name: Toyota Tundra
make name: Toyota Tundra
Processing data for: http://carsalesbase.com/us-car-sales-data/bmw/bmw-7-series
file name: BMW 7-series
make name: BMW 7-series
Processing data for: http://carsalesbase.com/us-car-sales-data/fiat/fiat-500/
file name: Fiat 500
make name: Fiat 500
Processing data for: http://carsalesbase.com/us-car-sales-data/jaguar/jaguar-xj
file nam

In [14]:
test_pd = pd.read_csv('sales-csvs/test-car.csv', sep=",")
test_pd.dtypes

Make                 object
Month                object
2015                  int64
2016                  int64
2017                  int64
2018                  int64
BMW 7-series 2019    object
dtype: object

In [20]:
year_sale

,Make,Month,2015,2016,2017,2018,Jeep Wrangler 2019
2,Jeep Wrangler,February,"12,911","13,234","13,641","15,936",15.001
3,Jeep Wrangler,March,"17,524","17,710","16,336","27,829",empty
4,Jeep Wrangler,April,"18,849","19,003","18,841","29,776",empty
5,Jeep Wrangler,May,"22,324","19,551","19,931","25,102",empty
6,Jeep Wrangler,June,"19,159","20,060","18,839","23,110",empty
7,Jeep Wrangler,July,"19,320","18,741","18,698","21,308",empty
8,Jeep Wrangler,August,"18,160","15,290","16,808","20,168",empty
9,Jeep Wrangler,September,"17,583","14,255","15,714","15,983",empty
10,Jeep Wrangler,October,"15,751","14,469","13,391","13,318",empty
11,Jeep Wrangler,November,"13,847","12,957","13,289","15,963",empty


### Add code to capture each table. 
* Previous table is showed each time code iterates to next car
* Need to handle tables with different date values
    * e.g. some tables only have 2018 data, some have 2019 data

In [65]:
tables 

[            0                   1                   2                   3  \
 0         NaN  Jeep Wrangler 2015  Jeep Wrangler 2016  Jeep Wrangler 2017   
 1     January              11.683              10.797              11.334   
 2    February              12.911              13.234              13.641   
 3       March              17.524              17.710              16.336   
 4       April              18.849              19.003              18.841   
 5         May              22.324              19.551              19.931   
 6        June              19.159              20.060              18.839   
 7        July              19.320              18.741              18.698   
 8      August              18.160              15.290              16.808   
 9   September              17.583              14.255              15.714   
 10    October              15.751              14.469              13.391   
 11   November              13.847              12.957          

### Clean data 

In [ ]:
# take 1 of 4 in list
yearly_table.columns = tables[0].iloc[0]

# fill null values
yearly_table_2 = yearly_table.fillna('null')

# Create Month Column
yearly_table_2.rename(columns={ yearly_table_2.columns[0]: "Month"}, inplace=True)

# Capture make_name by stripping off year
make_name = yearly_table_2.columns[1][:-4].strip()

# Remove duplicate row
yearly_final = yearly_table_2.iloc[2:]

# Create 'Make Column'
yearly_final.insert(loc = 0, column = 'Make', value = make_name, allow_duplicates=False)

# Create empty header list varible
header = []

# Create new column headers using years (2015,2016,2017,etc.)
new_columns = yearly_final.columns[2:].values
for column in new_columns:
    header.append(column[-4:])
    
# Create new dataframe with year columns and values
yearly = yearly_final.rename(columns={ yearly_final.columns[2]: header[0],
                                      yearly_final.columns[3]: header[1],
                                      yearly_final.columns[4]:header[2],
                                      yearly_final.columns[5]:header[3]
                                      
                                  })
# make a copy dataframe copy
year_sale = yearly
# replace decimal point values with comma to denote thousands (U.S. units of cars sold)
year_sale['2015'] = [x.replace('.', ',') for x in year_sale['2015']]
year_sale['2016'] = [x.replace('.', ',') for x in year_sale['2016']]
year_sale['2017'] = [x.replace('.', ',') for x in year_sale['2017']]
year_sale['2018'] = [x.replace('.', ',') for x in year_sale['2018']]

### Export tables to csv

### Use csv files to load mysql database

In [12]:
# CREATE DATABASE connection
#password = os.environ['MY_SQL_PW']
rds_connection_string = f"root:{password}@127.0.0.1/cardb_test"
engine = create_engine(f'mysql://{rds_connection_string}')

In [13]:
Base = declarative_base()
session = Session(bind=engine)

# Object relational mapping for our table
class CarSales(Base):
    __tablename__ = 'car_sales'
    id = Column(Integer, primary_key=True)
    model = Column(String(255))
    month = Column(String(50))
    Year_2015 = Column(Integer)
    Year_2016 = Column(Integer)
    Year_2017 = Column(Integer)
    Year_2018 = Column(Integer)
    Year_2019 = Column(Integer)

In [ ]:
c = 0
maxRows = 10000
start_time = datetime.datetime.now()
for index, row in yearly.iterrows():
    record = CarSales()
    record.model = row['Model']
    record.month = row['Month']
    record.Year_2015 = row['2015']
    record.Year_2016 = row['2016']
    record.Year_2017 = row['2017']
    record.Year_2018 = row['2018']
    record.Year_2019 = row['2019']
    session.add(record)
    session.commit()
    c = c + 1
    if c >= maxRows: break

end_time = datetime.datetime.now()
print(end_time - start_time)